In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pygsp import graphs, filters, plotting
import pygsp as pg
from numpy.random import default_rng
from numpy.random import randint
from random import sample
import random
import cvxpy as cp

plotting.BACKEND = 'matplotlib'
plt.rcParams['figure.figsize'] = (10, 5)

In [2]:
def createGraph(Nodes,i,graphType):
    while True:
        s = np.array(sample(range(1, 10000), 1))
        G = pg.graphs.Sensor(N=Nodes, seed = s)
        typeName = 'Sensor'
            
        if (G.is_connected() and not(G.is_directed())):
            break
    if i==0:        
        print(typeName)
           
    G.compute_fourier_basis()
    W = np.random.uniform(size=(2,1))
    x =  G.U[:,1:3]@W
    x = x/np.max(np.abs(x))
    eigenvalue = G.e
    print(eigenvalue[1:4])

    return G,x

In [3]:
def computeMainSignal(signals,k,N):
    x = np.zeros((N,1))
    for i in range(k):
        x = x + signals[:,i].reshape(N,1)
    return x

In [4]:
def computeSmoothness(signals,graphs,k):
    smoothness = []
    sumOverSmoothness = 0
    for i in range(k):
        smoothness.append(signals[:,i].T@graphs[i].L@signals[:,i])
        sumOverSmoothness = sumOverSmoothness + smoothness[i]
    return sumOverSmoothness,smoothness

In [5]:
def decomposeGraphSignal(mainSignal,signals,graphs,k,N):
    A = np.tile(np.identity(N), (1, k))
    
    H = np.zeros((N*k,N*k))
    for i in range(k):
        g = graphs[i].L
        H[i*N:(i+1)*N,i*N:(i+1)*N] = g.toarray()
        
        
    X = cp.Variable((N*k,1))
    objective = cp.Minimize(cp.quad_form(X, H)) 
    
    constraints = [ A @ X == mainSignal]
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=True)
    
    temp = X.value
    X_hat = temp.reshape(N,k, order='F')
    
    return X_hat

In [6]:
def makeMeanZero(signals,k):
    for i in range(k):
            signals[:,i] = signals[:,i] - np.mean(signals[:,i])
            
    return signals

In [7]:
def calculateSNR(est_signals,signals,k):
    SNR = np.zeros((1,k))
    for i in range(k):
        SNR[0,i]=(10*np.log10(np.linalg.norm(signals[:,i])**2/np.linalg.norm(signals[:,i]-est_signals[:,i])**2))
    return SNR

In [8]:
N = 450  # number of nodes
k = 5    # number of graphs
np.random.seed(0)
random.seed(0)
iteration = 200
av_snr = np.zeros((1,k))

for itera in range(1,iteration+1):
    print('############################################# iteration = ',itera,' ########################################')
    # create graph and graph signals
    graphType = randint(1, 3, 1)
    graph_list = []
    signals = np.zeros((N,k))
    for i in range(k):
        G , x = createGraph(N,i,graphType)
        graph_list.append(G)
        signals[:,[i]] = x

    # calculate the main signal : x = x1+x2+...+x4
    mainSignal = computeMainSignal(signals,k,N)
    mainSignal = mainSignal - np.mean(mainSignal)

    # decompose graph signal
    estimated_signals = decomposeGraphSignal(mainSignal,signals,graph_list,k,N)

    # calculate graph signals' smoothness
    sumOverSmoothness,smoothness = computeSmoothness(signals,graph_list,k)
    print('smoothness:')
    print(smoothness)
    print('sum over smoothness:',sumOverSmoothness)

    # calculate estimated graph signals' smoothness
    sumOverSmoothness_est,smoothness_est = computeSmoothness(estimated_signals,graph_list,k)
    print('smoothness_est:')
    print(smoothness_est)
    print('sum over smoothness_est:',sumOverSmoothness_est)

    # make mean of the estimated signals zero to remove DC values
    estimated_signals = makeMeanZero(estimated_signals,k)

    # calculate SNRs
    SNRs = calculateSNR(estimated_signals,signals,k)
    print('SNR:',SNRs)
    av_snr = av_snr + SNRs/iteration

    # compute objective values
    objectiveValue = sumOverSmoothness
    objectiveValue_est = sumOverSmoothness_est
    print('objectiveValue:',objectiveValue)
    print('objectiveValue_est:',objectiveValue_est)

    # compute error
    error = np.linalg.norm(mainSignal-estimated_signals@np.ones((k,1)))
    print('error:',error)
    
    
print('av_snr: ',av_snr)
print('total_av_snr: ',np.mean(av_snr))

############################################# iteration =  1  ########################################
Sensor
[0.03158285 0.07100325 0.10882611]
[0.04200162 0.0523515  0.08870145]
[0.05083379 0.0551267  0.08920562]
[0.0559381  0.05989505 0.10349108]
[0.0506435  0.07293445 0.11618803]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2250, constraints m = 450
          nnz(P) + nnz(A) = 17434
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: 

  50   2.6953e+01   4.86e-09   5.71e-08   1.00e-01   9.29e-01s
plsh   2.6953e+01   5.50e-16   1.08e-14   --------   1.70e+00s

status:               solved
solution polish:      successful
number of iterations: 50
optimal objective:    26.9531
run time:             1.70e+00s
optimal rho estimate: 1.31e-02

smoothness:
[1.9299545204492077, 8.658219750770286, 8.852005478513409, 3.803079198844797, 4.7419171887423195]
sum over smoothness: 27.985176137320018
smoothness_est:
[2.021675142966587, 8.248145624166012, 8.380839306441004, 3.783943845558885, 4.518522936842981]
sum over smoothness_est: 26.95312685597547
SNR: [[24.48456357 28.26291669 27.4651169  27.96160172 25.51099792]]
objectiveValue: 27.985176137320018
objectiveValue_est: 26.95312685597547
error: 5.3159593350246076e-15
############################################# iteration =  6  ########################################


2021-12-03 14:50:56,313:[WARNING](pygsp.graphs.sensor.__init__): Graph is not connected.


Sensor
[0.0498458  0.07280417 0.11640954]
[0.04891071 0.05268543 0.08409226]
[0.05853166 0.06303884 0.11725142]
[0.06006478 0.0652168  0.1129662 ]
[0.04113579 0.05346677 0.07612131]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2250, constraints m = 450
          nnz(P) + nnz(A) = 17546
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rh

  50   2.6700e+01   5.81e-09   5.47e-08   1.00e-01   1.14e+00s
plsh   2.6700e+01   5.94e-16   1.09e-14   --------   1.84e+00s

status:               solved
solution polish:      successful
number of iterations: 50
optimal objective:    26.6998
run time:             1.84e+00s
optimal rho estimate: 1.19e-02

smoothness:
[7.397573980449996, 6.448172390201196, 2.7296584753592645, 3.5395496125270376, 7.393493394933297]
sum over smoothness: 27.50844785347079
smoothness_est:
[7.1267227637336585, 6.1538449895867995, 2.693702824710842, 3.5762595180521557, 7.1492393439996755]
sum over smoothness_est: 26.69976944008313
SNR: [[29.74652536 28.23012079 26.08740062 29.84510097 29.76126914]]
objectiveValue: 27.50844785347079
objectiveValue_est: 26.69976944008313
error: 5.297891211432144e-15
############################################# iteration =  11  ########################################
Sensor
[0.06160342 0.07630633 0.13073751]
[0.06283748 0.07133387 0.11696337]
[0.06075485 0.06650353 0.10789399

...

2021-12-03 15:04:52,173:[WARNING](pygsp.graphs.sensor.__init__): Graph is not connected.


Sensor
[0.03671767 0.06486174 0.11748764]
[0.03695601 0.04936389 0.10801765]
[0.04531861 0.0679215  0.11672538]
[0.05416824 0.06582305 0.12428376]
[0.03902722 0.05120976 0.08356014]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2250, constraints m = 450
          nnz(P) + nnz(A) = 17693
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rh

2021-12-03 15:05:12,168:[WARNING](pygsp.graphs.sensor.__init__): Graph is not connected.


Sensor
[0.03339176 0.0551225  0.0830865 ]
[0.05425725 0.07438369 0.10881669]
[0.03226623 0.06377213 0.08378716]
[0.06222808 0.07029518 0.11032811]
[0.0461533  0.05665824 0.10473035]
-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2250, constraints m = 450
          nnz(P) + nnz(A) = 17501
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on, time_limit: off

iter   objective    pri res    dua res    rh

  50   2.6870e+01   5.34e-09   4.99e-08   1.00e-01   1.13e+00s
plsh   2.6870e+01   4.54e-16   8.23e-15   --------   1.90e+00s

status:               solved
solution polish:      successful
number of iterations: 50
optimal objective:    26.8699
run time:             1.90e+00s
optimal rho estimate: 1.26e-02

smoothness:
[7.252699241071836, 5.498348244165183, 4.216689462026553, 6.115526532060938, 4.889309388745554]
sum over smoothness: 27.972572868070067
smoothness_est:
[6.867817240613819, 5.280815535112434, 4.219662933897463, 5.794377924856437, 4.707234036094483]
sum over smoothness_est: 26.869907670574637
SNR: [[26.51533988 26.48755913 28.74449589 26.25565033 25.7673442 ]]
objectiveValue: 27.972572868070067
objectiveValue_est: 26.869907670574637
error: 5.0435040047601075e-15
############################################# iteration =  199  ########################################
Sensor
[0.04086707 0.06457487 0.09101912]
[0.05049677 0.06885452 0.1053207 ]
[0.06024644 0.06977    0.11210404